In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import os
import pydicom
import SimpleITK as sitk
from radiomics import featureextractor


def load_dicom_series(directory):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(directory)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    return image

def load_dicom_file(file_path):
    return sitk.ReadImage(file_path)

def resample_image_to_reference(image, reference_image):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(reference_image)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampler.SetTransform(sitk.Transform())
    resampler.SetOutputSpacing(reference_image.GetSpacing())
    resampler.SetSize(reference_image.GetSize())
    resampler.SetOutputDirection(reference_image.GetDirection())
    resampler.SetOutputOrigin(reference_image.GetOrigin())
    return resampler.Execute(image)

paths_df = pd.read_csv('df_patients.csv')
paths_df

,Unnamed: 0,Patient,Path_ct,Path_seg
0,0,LIDC-IDRI-0001,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
1,1,LIDC-IDRI-0002,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
2,2,LIDC-IDRI-0003,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
3,3,LIDC-IDRI-0004,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
4,4,LIDC-IDRI-0005,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
...,...,...,...,...
870,870,LIDC-IDRI-1008,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
871,871,LIDC-IDRI-1009,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
872,872,LIDC-IDRI-1010,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
873,873,LIDC-IDRI-1011,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...


In [105]:
pat_list = pd.read_excel("tcia-diagnosis-data-2012-04-20.xls")['TCIA Patient ID'].tolist()

#filter all rows with patient id in pat_list
paths_df = paths_df[paths_df['Patient'].isin(pat_list)]
paths_df.drop(columns=['Unnamed: 0'], inplace=True)
paths_df.reset_index(drop=True, inplace=True)
paths_df

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_12776\1755107242.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paths_df.drop(columns=['Unnamed: 0'], inplace=True)


,Patient,Path_ct,Path_seg
0,LIDC-IDRI-0068,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
1,LIDC-IDRI-0072,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
2,LIDC-IDRI-0088,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
3,LIDC-IDRI-0090,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
4,LIDC-IDRI-0091,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
...,...,...,...
137,LIDC-IDRI-0994,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
138,LIDC-IDRI-1002,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
139,LIDC-IDRI-1004,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...
140,LIDC-IDRI-1010,G:/Dokumente/MedicalIM/Lung_Cancer/manifest-16...,['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manif...


In [106]:
list_of_seg_paths = []
for path in paths_df["Path_seg"]:
    path_seg1 = ast.literal_eval(path)
    temp = []
    for i in path_seg1:
        files = os.listdir(i)[0]
        path_seg_full = f'{i}/{files}'
        temp.append(path_seg_full)
    list_of_seg_paths.append(temp)

list_of_seg_paths[10]

['G:/Dokumente/MedicalIM/Lung_Cancer_Seg/manifest-1585232716547/LIDC-IDRI/LIDC-IDRI-0138/01-01-2000-NA-CT CHEST W CONT-37148/103.000000-Segmentation of Nodule 1 - Annotation 98644-80625/1-1.dcm']

In [107]:
# Initialize feature extractor (assuming you have a params.yaml)
params = 'C:/Users/Gabriel/OneDrive/Dokumente/GitHub/Medical_image/git_extraction/params/qa4iqi_params.yml'
extractor = featureextractor.RadiomicsFeatureExtractor(params)

list_radiomics = []

for ct in range(10):#range(len(paths_df["Path_ct"])):
    image = load_dicom_series(paths_df["Path_ct"][ct])
    for seg in range(len(list_of_seg_paths[ct])):
        mask = load_dicom_file(list_of_seg_paths[ct][seg])
        resampled_mask = resample_image_to_reference(mask, image)
        features = extractor.execute(image, resampled_mask, label=255)
        list_radiomics.append(features)
#ct_image = load_dicom_series(paths_df["Path_ct"][0])
#mask = load_dicom_file(path_seg_full)

KeyboardInterrupt: 

In [ ]:
for feature_name, feature_value in list_radiomics[10].items():
    print(f"{feature_name}: {feature_value}")

diagnostics_Versions_PyRadiomics: 0+unknown
diagnostics_Versions_Numpy: 1.26.2
diagnostics_Versions_SimpleITK: 2.1.1.2
diagnostics_Versions_PyWavelet: 1.2.0
diagnostics_Versions_Python: 3.10.13
diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}
diagnostics_Configuration_EnabledImageTypes: {'Original': {}}
diagnostics_Image-original_Hash: 1f4847eac70bc5c09b160de0e751ad601e0f690c
diagnostics_Image-original_Dimensionality: 3D
diagnostics_Image-original_Spacing: (0.820312, 0.820312, 2.5)
diagnostics_Image-original_Size: (512, 512, 140)
diagnostics_Image-original_Mean: -802.6419776371547
diagnostics_Image-original_Minimum: -2048.0
diagno

In [ ]:


# Resample mask to match CT image
resampled_mask = resample_image_to_reference(mask, ct_image)





for feature_name, feature_value in features.items():
    print(f"{feature_name}: {feature_value}")

diagnostics_Versions_PyRadiomics: 0+unknown
diagnostics_Versions_Numpy: 1.26.2
diagnostics_Versions_SimpleITK: 2.1.1.2
diagnostics_Versions_PyWavelet: 1.2.0
diagnostics_Versions_Python: 3.10.13
diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}
diagnostics_Configuration_EnabledImageTypes: {'Original': {}}
diagnostics_Image-original_Hash: e4e20cc4719f15703e30af6acffb6e06df4da0d5
diagnostics_Image-original_Dimensionality: 3D
diagnostics_Image-original_Spacing: (0.703125, 0.703125, 2.5)
diagnostics_Image-original_Size: (512, 512, 133)
diagnostics_Image-original_Mean: -826.9439289121699
diagnostics_Image-original_Minimum: -2048.0
diagno

In [ ]:
features.items()


odict_items([('diagnostics_Versions_PyRadiomics', '0+unknown'), ('diagnostics_Versions_Numpy', '1.26.2'), ('diagnostics_Versions_SimpleITK', '2.1.1.2'), ('diagnostics_Versions_PyWavelet', '1.2.0'), ('diagnostics_Versions_Python', '3.10.13'), ('diagnostics_Configuration_Settings', {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 255, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None}), ('diagnostics_Configuration_EnabledImageTypes', {'Original': {}}), ('diagnostics_Image-original_Hash', 'e4e20cc4719f15703e30af6acffb6e06df4da0d5'), ('diagnostics_Image-original_Dimensionality', '3D'), ('diagnostics_Image-original_Spacing', (0.703125, 0.703125, 2.5)), ('diagnostics_Image-original_Size', (512, 512, 133)), ('diagnostics_Image-

In [ ]:
#list_radiomics to dataframe
df_radiomics = pd.DataFrame(list_radiomics)
#write to csv
df_radiomics.to_csv('df_radiomics.csv')

